In [2]:
%load_ext autoreload 
%autoreload 1
import os
# os.environ['SNORKELDB'] = 'sqlite:///brat-import.db'
from set_env import set_env
set_env()
from snorkel import SnorkelSession
import snorkel.contrib.brat as brt
from snorkel.parsers import StanfordCoreNLPServer

from snorkel.models import Candidate, Document, candidate_subclass, GoldLabel

session = SnorkelSession()

In [3]:
corenlp_server = StanfordCoreNLPServer(version='3.6.0', split_newline=False, num_threads=1)

In [4]:
brat = brt.Brat(session)

In [5]:
# Only run on documents which have annotations

# from snorkel.parser import TextDocPreprocessor, CorpusParser
# import multiprocessing
# doc_preprocessor = TextDocPreprocessor("brat_data/test_labeled_docs/*.txt", encoding="utf-8")
# corpus_parser = CorpusParser()
# corpus_parser.apply(doc_preprocessor, parralelism=multiprocessing.cpu_count())

In [6]:
input_dir = "brat_data/test_labeled_docs/"
brat.import_project(input_dir, annotations_only=False, annotator_name='brat', num_threads=1, parser=corenlp_server)

In [7]:
pheno_spans = brat.explore()
print(len(pheno_spans))
print len([pheno_span for pheno_span in pheno_spans if len(pheno_span) > 1])
for p in pheno_spans:
    print p

544
91
[TemporarySpan("", sentence=459557, chars=[62,131], words=[0,0])]
[TemporarySpan("trong expression in meiocytes as revealed by RNA in situs.", sentence=459555, chars=[15,190], words=[2,12])]
[TemporarySpan("", sentence=450620, chars=[15,54], words=[1,1])]
[TemporarySpan("", sentence=450608, chars=[108,181], words=[4,4])]
[TemporarySpan("", sentence=450612, chars=[26,99], words=[1,1])]
[TemporarySpan("", sentence=450592, chars=[37,65], words=[1,1])]
[TemporarySpan("", sentence=450869, chars=[276,298], words=[1,1])]
[TemporarySpan("rocessing factor is S1P", sentence=449818, chars=[7,29], words=[2,5])]
[TemporarySpan("severely developmentally compromised", sentence=461436, chars=[187,222], words=[27,29])]
[TemporarySpan("downward-oriented siliques", sentence=425366, chars=[119,144], words=[21,22])]
[TemporarySpan("", sentence=411950, chars=[168,209], words=[0,0])]
[TemporarySpan("", sentence=411946, chars=[102,185], words=[10,10])]
[TemporarySpan(".
", sentence=410427, chars=[0,76]

In [8]:
from snorkel.models import Document, Sentence, Span

In [9]:
PhenoPair = candidate_subclass('Phenotypes', ['descriptor', 'entity'])

In [26]:
import collections
from sqlalchemy import and_
from snorkel.models import StableLabel, GoldLabel, GoldLabelKey

jaccard_cutoff = 0.3

ak = session.query(GoldLabelKey).filter(GoldLabelKey.name == 'gold').first()
if ak is None:
    ak = GoldLabelKey(name='gold')
    session.add(ak)
    session.commit()

candidates = session.query(PhenoPair).filter(PhenoPair.split == 0).all()
print(len(candidates), "candidates")
sentence_to_pheno = collections.defaultdict(list)

for pheno_span in pheno_spans:
    # pheno_span is a list of fragments, where each fragment is a temporary span
    sent_id = pheno_span[0].sentence.id
    sentence_to_pheno[sent_id].append(pheno_span)

num = 0
num_matches = 0
num_sentences = 0
num_gold_phenos = 0
num_zero_matches = 0
avg = 0

labeled = {}

for sentence_id, gold_phenos in sentence_to_pheno.iteritems():
    if num%1000==0:print num
    sentence_match = False
    results = session.query(Span, PhenoPair).filter(and_(Span.sentence_id == sentence_id, PhenoPair.descriptor_id == Span.id)).all()
    if (len(results) == 0):
        num_zero_matches += 1
#         print "sentence", sentence_id
#         for gold_pheno in gold_phenos:
#             print(gold_pheno.get_span())
#         print
        
    for gold_pheno_fragments in gold_phenos:
        matched = False
        for span, cand_pheno in results:
            # Build word indice set for each of the gold phenotype, and the candiate phenotype
            cand_words = set()
            gold_words = set()
            # combine both entities (modifier and attribute) into one "phenotype"
            [cand_words.update(xrange(span.get_word_start(), span.get_word_end()+1)) for span in cand_pheno.get_contexts()]
            
            for gold_pheno in gold_pheno_fragments:
                gold_words.update(xrange(gold_pheno.get_word_start(), gold_pheno.get_word_end()+1))
            intersect = gold_words.intersection(cand_words)
            jaccard_score = float(len(intersect)) / len(cand_words.union(gold_words))
            
            val = -1
            if jaccard_score > jaccard_cutoff:
                matched = True
                sentence_match = True
                avg += jaccard_score
                num += 1
                val = 1
#                 print(jaccard_score)
#                 print(gold_pheno_fragments)
#                 print(cand_pheno)
#             print(cand_pheno)
#                 print("\n")

#             context_stable_ids = '~~'.join([c.stable_id for c in cand_pheno.get_contexts()])
            #session.add(StableLabel(context_stable_ids=context_stable_ids, annotator_name='gold', value=val, split=0))#cand_pheno.split)) 
            label = session.query(GoldLabel).filter(GoldLabel.key == ak).filter(GoldLabel.candidate == cand_pheno).first()
            if label is None:
                session.add(GoldLabel(candidate=cand_pheno, key=ak, value=val))
            elif label.value == -1 and val == 1:
                label.value = val

        num_gold_phenos += 1
        if matched:
            num_matches += 1
    if sentence_match:
        num_sentences += 1
        
session.commit()

print("avg", avg / num)
print("num matches", num_matches)
recall = float(num_matches) / num_gold_phenos
print("num gold phenos", num_gold_phenos)
num_sentences_tagged = len(sentence_to_pheno.keys())
print("num_sentences_tagged by tanya", num_sentences_tagged)
print("sentences", float(num_sentences) / num_sentences_tagged)
print("num sents missed", float(num_zero_matches) / num_sentences_tagged)
print("recall", recall)

(42946, 'candidates')
0
0
0
('avg', 0.5080205650192784)
('num matches', 162)
('num gold phenos', 544)
('num_sentences_tagged by tanya', 489)
('sentences', 0.310838445807771)
('num sents missed', 0.49284253578732107)
('recall', 0.2977941176470588)


In [29]:
# each entity/relation type is assigned to a different split
#for i in range(0,1):

print len(candidates)
for i,c in enumerate(candidates):
    #label = session.query(GoldLabel).filter(GoldLabel.key == ak).filter(GoldLabel.candidate == c).first()
    #if label is not None and label.value == 1: print c, label
    print type(c).type, c
    if i > 5:
        break
print
    
sents = list(set([pheno_span[0].sentence.id for pheno_span in pheno_spans]))
candidates = [c for c in candidates if c[0].sentence.id in sents]
    
# ProteinComponent relations
#candidates = session.query(PhenoPair).filter(PhenoPair.split == 0).all()

42946
Phenotypes.type Phenotypes(Span("turned green", sentence=403502, chars=[120,131], words=[21,22]), Span("lower in albino leaves", sentence=403502, chars=[28,49], words=[6,9]))
Phenotypes.type Phenotypes(Span("turned green", sentence=403502, chars=[120,131], words=[21,22]), Span("leaves", sentence=403502, chars=[113,118], words=[20,20]))
Phenotypes.type Phenotypes(Span("turned green", sentence=403502, chars=[120,131], words=[21,22]), Span("decreased in pale-green leaves while they", sentence=403502, chars=[52,92], words=[11,16]))
Phenotypes.type Phenotypes(Span("increased", sentence=403502, chars=[94,102], words=[17,17]), Span("lower in albino leaves", sentence=403502, chars=[28,49], words=[6,9]))
Phenotypes.type Phenotypes(Span("increased", sentence=403502, chars=[94,102], words=[17,17]), Span("leaves", sentence=403502, chars=[113,118], words=[20,20]))
Phenotypes.type Phenotypes(Span("increased", sentence=403502, chars=[94,102], words=[17,17]), Span("decreased in pale-green leaves

### this is how we can selectively only look at the labeled sentences for training/lf functions

In [34]:
sents = set([pheno_span[0].sentence.id for pheno_span in pheno_spans])
print len(sents)
c = [p for p in session.query(PhenoPair).filter(PhenoPair.split == 0).all() if p[0].sentence.id in sents]

489


In [31]:
from snorkel.viewer import SentenceNgramViewer

sv = SentenceNgramViewer(candidates, session=session, n_per_page=6, height=400,
                         annotator_name='gold')
sv

<IPython.core.display.Javascript object>

In [ ]:
# Select spans from brat
# Select candidates from phenos
# For each candidate in phenos
# 